# Introducing Supriya

A Python API for SuperCollider

https://github.com/josiah-wolf-oberholtzer/supriya

## Supriya lets you...

- boot and communicate with SuperCollider's `scsynth` synthesis server

- construct and compile SynthDef unit generator graphs in native Python code

- build and control graphs of synthesizers and synthesizer groups

- explicitly object-model `scsynth`-specific OSC commands  via `Request` and `Response` classes

- compile non-realtime synthesis scores via Supriya's `Session` class

- write patterns for realtime or non-realtime synthesis

A lot of the same stuff you do with `sclang` and `scide`, just in Python instead.

## About the author

- A composer and programmer
  - https://github.com/josiah-wolf-oberholtzer
  - https://soundcloud.com/josiah-wolf-oberholtzer/in-the-tall-grasses

- PhD from Harvard in Music Composition, specializing in massively multi-channel tape music and symbolic computer-assisted composition

- Core contributor to Abjad (https://http://abjad.mbrsi.org/), a Python API for LilyPond

- Engineering team lead at Capital One, managing a group developing serverless machine learning applications for hotel reservation arbitrage

- Spent way too much time using and teaching Max

## Hold up, what's Python?

## OK, but why make another `scsynth` client?

- To take advantage of language features and libraries not available in `sclang`
- To explore language features and libraries in your chosen language you might not otherwise interact with
- To better understand how `sclang` and `scsynth` interact
- Just For Fun™
- Because you're stubborn

## No, really, why?

- To make massively multichannel fixed media pieces in my preferred language
- To have all audio materials modeled in code
- To create parity between the experiences of realtime experimentation and non-realtime composing
- To allow layers of NRT material to reference one another in an object-oriented way
- To allow for fully-reproducible (re-)rendering of NRT scores
- To allow scores and related tools to be fully tested

## Hello World

In [1]:
from supriya import Server, Synth

In [2]:
server = Server()
server.boot()

<Server: udp://127.0.0.1:57751, 8i8o>

In [3]:
synth = Synth()
synth.allocate()

<+ Synth: 1000>

In [4]:
print(server)

NODE TREE 0 group
    1 group
        1000 default
            out: 0.0, amplitude: 0.1, frequency: 440.0, gate: 1.0, pan: 0.5


In [6]:
synth.release()

<- Synth: ???>

In [7]:
server.quit()

<Server: offline>

## Hello World (a little more complicated)

In [8]:
from supriya import Bus, Group, Server, Synth

In [9]:
server = Server().boot()

In [10]:
bus = Bus.control().allocate()
bus.set(0.5)

In [11]:
group = Group().allocate()
for i in range(1, 10):
    synth = Synth(amplitude=bus, frequency=111 * i)
    _ = synth.allocate(target_node=group)

In [12]:
bus.set(1)

In [13]:
group.controls["gate"] = 0

In [14]:
server.quit()

<Server: offline>

# Node and Message Aggregation

### Nodes (synths and groups) can be aggregated just like Python lists

In [ ]:
synth_a = Synth(frequency=444)
synth_b = Synth(frequency=555)
synth_c = Synth(frequency=666)
inner_group = Group()
outer_group = Group()
inner_group.append(synth_b)
outer_group.extend([synth_a, inner_group, synth_c])

### Allocation, deallocation, ordering are auto-bundled

In [ ]:
server = Server().boot()
_ = synth_a.synthdef.allocate()  # manual allocation

In [ ]:
with server.osc_io.capture() as osc_capture:
    server.default_group.append(outer_group)  # everything allocated at once!

In [ ]:
for osc_message in osc_capture.sent_messages:
    print(repr(osc_message))

In [ ]:
print(server)

In [ ]:
with server.osc_io.capture() as osc_capture:
    inner_group.extend([synth_a, synth_c])

In [ ]:
for osc_message in osc_capture.sent_messages:
    print(repr(osc_message))

### Freed subtrees do not lose their structure

In [ ]:
print(server)

In [ ]:
outer_group.free()

In [ ]:
print(outer_group)

### Message aggregation knows about completion messages

In [ ]:
synth_a.synthdef.free()  # this is really the default synthdef
with server.osc_io.capture() as osc_capture:
    outer_group.allocate()

Oh boy...

In [ ]:
for osc_message in osc_capture.sent_messages:
    print(repr(osc_message))

In [ ]:
server.quit()

## Graphviz Everything

In [ ]:
%reload_ext supriya.ext.ipython

In [ ]:
import supriya
_ = supriya.graph(synthdefs.default)

## Feature: Realtime Server Node Tree Model
## Feature: OSC Command Aggregation
## Feature: Requests and Responses
## Feature: SynthDef Builders
## Feature: Graphviz Everything
## Feature: SynthDef Factories
## Feature: Non-realtime Session Model
## Feature: Non-realtime Session "Renderables"
## Feature: NRT Dependency Tree (turtles all the way down)
## Feature: (N)RT Patterns
## Feature: CLI Tooling and NRT Projects